<h1> Analisis inicial de datos </h1>

<h4> Contenido Notebook</h4>
<p>1. Setup </p>
<p>2. Funciones Reutilizables </p>
<p>3. Exploración de Clientes </p>
<p>4. Exploración de Moras </p>
<p>5. Exploración Canales (Deep) </p>
<p>6. Exploración  Gestiones </p>
<p>7. Exploración Excedentes </p>
<p>8. Exploración Tanque Movimientos </p>
<p>9. Conclusiones Globales </p>

<h5>1. Setup </h5>

In [1]:
import duckdb
import pandas as pd
from pathlib import Path
from collections import Counter, defaultdict

DB_PATH = Path.cwd().parent / "database" / "analytics.duckdb"
DB_PATH

con = duckdb.connect(str(DB_PATH))

<h5>2. Función Reutilizable </h5>

In [2]:
def explore_table_schema(table_name):

    print(f"\n{'='*60}")
    print(f"Explorando tabla: {table_name}")
    print(f"{'='*60}")

    df_cols = con.execute(f"""
        DESCRIBE raw.{table_name}
    """).fetchdf()

    cols = df_cols["column_name"].tolist()

    print(f"\nTotal columnas: {len(cols)}")

    prefixes = [c.split("_")[0] for c in cols]
    print("\nTop prefijos:")
    print(Counter(prefixes).most_common(10))

    suffixes = [c.split("_")[-1] for c in cols]
    print("\nTop sufijos:")
    print(Counter(suffixes).most_common(10))

    print("\nEjemplo columnas:")
    print(cols[:20])

    return cols


<h5>3. Exploración de Clientes </h5>

In [3]:
clientes_cols = explore_table_schema("clientes")


Explorando tabla: clientes

Total columnas: 4

Top prefijos:
[('num', 1), ('obl17', 1), ('f', 1), ('var', 1)]

Top sufijos:
[('doc', 1), ('obl17', 1), ('analisis', 1), ('rta', 1)]

Ejemplo columnas:
['num_doc', 'obl17', 'f_analisis', 'var_rta']


<h5>4. Exploración de Moras </h5>

In [4]:
moras_cols = explore_table_schema("moras")


Explorando tabla: moras

Total columnas: 12

Top prefijos:
[('moras', 9), ('num', 1), ('obl17', 1), ('f', 1)]

Top sufijos:
[('3m', 4), ('6m', 4), ('doc', 1), ('obl17', 1), ('analisis', 1), ('9m', 1)]

Ejemplo columnas:
['num_doc', 'obl17', 'f_analisis', 'moras_avg_mora_3m', 'moras_min_mora_3m', 'moras_max_mora_3m', 'moras_stddev_mora_3m', 'moras_avg_mora_6m', 'moras_min_mora_6m', 'moras_max_mora_6m', 'moras_stddev_mora_6m', 'moras_avg_mora_9m']


<h5>5. Exploración Canales (Deep) </h5>

In [5]:
canales_cols = explore_table_schema("canales")


Explorando tabla: canales

Total columnas: 4258

Top prefijos:
[('avg', 816), ('max', 816), ('stddev', 816), ('sum', 816), ('min', 789), ('trx', 202), ('num', 1), ('obl17', 1), ('f', 1)]

Top sufijos:
[('ult3', 1017), ('ult6', 1017), ('ult9', 1011), ('ult12', 1008), ('smmlv', 67), ('trf', 20), ('pag', 16), ('cmpr', 12), ('ob', 10), ('cr', 10)]

Ejemplo columnas:
['num_doc', 'obl17', 'f_analisis', 'trx_mnt_total', 'trx_mnt_total_smmlv', 'trx_cnt_total', 'trx_cnt_app_per_pag', 'trx_cnt_app_per_pag_ob', 'trx_cnt_app_per_tfy', 'trx_cnt_app_per_trf', 'trx_cnt_app_per_ava', 'trx_cnt_app_per_ut_c_cr', 'trx_cnt_app_pyme_trf', 'trx_cnt_bill_mvl_cmpr', 'trx_cnt_btn_bco_trf', 'trx_cnt_cajero_pag', 'trx_cnt_cajero_ava', 'trx_cnt_cajero_ret', 'trx_cnt_cajero_trf', 'trx_cnt_cr_bcrio_ret']


<p> Diccionario de canales</p>

In [7]:
CHANNEL_CATALOG = [
    {"canal": "gr",           "interpretacion": "gr"},
    {"canal": "cajero",       "interpretacion": "cajero"},
    {"canal": "app_per",      "interpretacion": "app personas"},
    {"canal": "pos",          "interpretacion": "pos"},
    {"canal": "suc_fis",      "interpretacion": "sucursal fisica"},
    {"canal": "svp",          "interpretacion": "sucursal virtual personas"},
    {"canal": "cr_bcrio",     "interpretacion": "cr_bcrio"},
    {"canal": "rec_e_v",      "interpretacion": "rec_e_v"},
    {"canal": "sve",          "interpretacion": "sucursal virtual empresa"},
    {"canal": "app_per_pag",  "interpretacion": "app_per_pag"},
    {"canal": "app_per_tfy",  "interpretacion": "app_per_tfy"},
    {"canal": "app_per_ut_c", "interpretacion": "app_per_ut_c"},
    {"canal": "app_pyme",     "interpretacion": "app pymes"},
    {"canal": "bill_mvl",     "interpretacion": "billetera movil"},
    {"canal": "btn_bco",      "interpretacion": "boton banco"},
    {"canal": "pse_emp",      "interpretacion": "pse empresas"},
    {"canal": "pse_per",      "interpretacion": "pse personas"},
    {"canal": "suc_fis_dep",  "interpretacion": "suc_fis_dep"},
    {"canal": "suc_tel_pag",  "interpretacion": "sucursal telefonica"},
    {"canal": "suc_tel_ut_c", "interpretacion": "suc_tel_ut_c"},
    {"canal": "sv_pyme",      "interpretacion": "sucursal virtual pymes"},
    {"canal": "sve_dvs",      "interpretacion": "sve_dvs"},
    {"canal": "sve_pag",      "interpretacion": "sve_pag"},
    {"canal": "sve_ut_c",     "interpretacion": "sve_ut_c"},
    {"canal": "svp_dvs",      "interpretacion": "svp_dvs"},
]

df_channels = pd.DataFrame(CHANNEL_CATALOG)

# Resolver raíz del proyecto: notebooks/ -> op_cobro/
PROJECT_ROOT = Path.cwd().parent  # si el notebook vive en op_cobro/notebooks

# Carpeta destino (la que creaste)
out_dir = PROJECT_ROOT / "data" / "process_files"
out_dir.mkdir(parents=True, exist_ok=True)

# Guardar CSV
out_path = out_dir / "channels_catalog.csv"
df_channels.to_csv(out_path, index=False, encoding="utf-8")

print("✅ Catálogo guardado en:", out_path)
df_channels


✅ Catálogo guardado en: c:\Users\serrios\01_prueba_analitico_4\op_cobro\data\process_files\channels_catalog.csv


,canal,interpretacion
0,gr,gr
1,cajero,cajero
2,app_per,app personas
3,pos,pos
4,suc_fis,sucursal fisica
5,svp,sucursal virtual personas
6,cr_bcrio,cr_bcrio
7,rec_e_v,rec_e_v
8,sve,sucursal virtual empresa
9,app_per_pag,app_per_pag


<h5>6. Exploración  Gestiones </h5>

In [8]:
gestiones_cols = explore_table_schema("gestiones")


Explorando tabla: gestiones

Total columnas: 83

Top prefijos:
[('avg', 20), ('min', 20), ('max', 20), ('stddev', 20), ('num', 1), ('obl17', 1), ('f', 1)]

Top sufijos:
[('3m', 20), ('6m', 20), ('9m', 20), ('12m', 20), ('doc', 1), ('obl17', 1), ('analisis', 1)]

Ejemplo columnas:
['num_doc', 'obl17', 'f_analisis', 'avg_cant_gestiones_3m', 'min_cant_gestiones_3m', 'max_cant_gestiones_3m', 'stddev_cant_gestiones_3m', 'avg_cant_rpc_3m', 'min_cant_rpc_3m', 'max_cant_rpc_3m', 'stddev_cant_rpc_3m', 'avg_cant_acuerdo_3m', 'min_cant_acuerdo_3m', 'max_cant_acuerdo_3m', 'stddev_cant_acuerdo_3m', 'avg_promesas_cumplidas_3m', 'min_promesas_cumplidas_3m', 'max_promesas_cumplidas_3m', 'stddev_promesas_cumplidas_3m', 'avg_maximo_rank_3m']


<h5>7. Exploración Excedentes </h5>

In [9]:
excedentes_cols = explore_table_schema("excedentes")


Explorando tabla: excedentes

Total columnas: 35

Top prefijos:
[('avg', 8), ('min', 8), ('max', 8), ('stddev', 8), ('num', 1), ('obl17', 1), ('f', 1)]

Top sufijos:
[('3m', 8), ('6m', 8), ('9m', 8), ('12m', 8), ('doc', 1), ('obl17', 1), ('analisis', 1)]

Ejemplo columnas:
['num_doc', 'obl17', 'f_analisis', 'avg_excedente_pago_3m', 'min_excedente_pago_3m', 'max_excedente_pago_3m', 'stddev_excedente_pago_3m', 'avg_excedente_pago_6m', 'min_excedente_pago_6m', 'max_excedente_pago_6m', 'stddev_excedente_pago_6m', 'avg_excedente_pago_9m', 'min_excedente_pago_9m', 'max_excedente_pago_9m', 'stddev_excedente_pago_9m', 'avg_excedente_pago_12m', 'min_excedente_pago_12m', 'max_excedente_pago_12m', 'stddev_excedente_pago_12m', 'avg_porc_pago_3m']


<h5> 8. Exploración Tanque Movimientos </h5>

In [10]:
tanque_cols = explore_table_schema("tanque_movimiento")


Explorando tabla: tanque_movimiento

Total columnas: 67

Top prefijos:
[('avg', 16), ('min', 16), ('max', 16), ('stddev', 16), ('num', 1), ('obl17', 1), ('f', 1)]

Top sufijos:
[('3m', 16), ('6m', 16), ('9m', 16), ('12m', 16), ('doc', 1), ('obl17', 1), ('analisis', 1)]

Ejemplo columnas:
['num_doc', 'obl17', 'f_analisis', 'avg_pago_debito_3m', 'min_pago_debito_3m', 'max_pago_debito_3m', 'stddev_pago_debito_3m', 'avg_pago_fisico_3m', 'min_pago_fisico_3m', 'max_pago_fisico_3m', 'stddev_pago_fisico_3m', 'avg_pago_virtual_3m', 'min_pago_virtual_3m', 'max_pago_virtual_3m', 'stddev_pago_virtual_3m', 'avg_pago_otros_3m', 'min_pago_otros_3m', 'max_pago_otros_3m', 'stddev_pago_otros_3m', 'avg_pago_debito_6m']


<h5> 9. Conclusiones Globales </h5>

A partir de la exploración de esquemas y nombres de columnas, se concluye lo siguiente:

La tabla raw.canales presenta retos importantes para su procesamiento debido a su alta dimensionalidad y a la heterogeneidad de medidas por canal (múltiples agregaciones, ventanas temporales y tipos de transacción)..

Es necesario condensar el comportamiento transaccional por cliente en señales analíticas útiles para el modelo, en lugar de consumir directamente todas las variables. 